# **Langauage Transcritption**

Managing voice records directly from google colab `using google transcription api` .

In [ ]:
##Installation of pyaudio library on Linux system
##Note: Another format is used when laoding such library on a windows system
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio
import pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
##Necessary importation for code running
!pip install webrtcvad 

In [ ]:
#Necessary importation for code running
!pip install SpeechRecognition

In [ ]:
#Necessary importation for code running
!pip install pydub

# **To DO (2)**

1. Complete the implementation of this section such that it can work locally omnon a computer.

In [ ]:
##Codes to be used in the when the jupyter file will be converted into pythin file
"""
import wave
import os, sys
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import tensorflow as tf
import webrtcvad

vad = webrtcvad.Vad()
vad.set_mode(2)
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 32000
CHUNK = 960
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "file.wav"

audio = pyaudio.PyAudio()

# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
"""

'\nimport wave\nimport os, sys\nimport librosa\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport librosa.display\nimport tensorflow as tf\nimport webrtcvad\n\nvad = webrtcvad.Vad()\nvad.set_mode(2)\nFORMAT = pyaudio.paInt16\nCHANNELS = 1\nRATE = 32000\nCHUNK = 960\nRECORD_SECONDS = 3\nWAVE_OUTPUT_FILENAME = "file.wav"\n\naudio = pyaudio.PyAudio()\n\n# start Recording\nstream = audio.open(format=FORMAT, channels=CHANNELS,\n                rate=RATE, input=True,\n                frames_per_buffer=CHUNK)\n'

Managing audio records on google colab. This will change while working locally. 
A developer will have to add a section for such implementation locally for test to be done, for those who will use the codes locally.

In [ ]:
# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript
from pydub import AudioSegment

wavfile = 'record.wav'
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

def verifyVoice(wavfile):
  """
  Function to verify the wave file record type
  Note: Not sure to be kept, seems not really necessary

  Input: None
  Output: Voice format
  """
  import fleep
  with open(wavfile, "rb") as file:
      info = fleep.get(file.read(128))

  print(info.extension)
  
  return info.extension[0]

def displayvoice(audio):
  import IPython.display as ipd
  ipd.display(ipd.Audio(audio))

def launchClientTranscript():
  """
  Function to convert voice directly.
  Note: This has been tested and works for the moment with english voice

  Input: None
  Output: voice format type, transcripted voice text
  """
  import speech_recognition as speech_recog
  rec = speech_recog.Recognizer() ##Google speech recognition API

  with open(wavfile, 'wb') as file:
       file.write(record()) ##saving the recorded file from audio
  flac_audio = AudioSegment.from_file(wavfile)
  flac_audio.export("record.wav", format="wav") ##Giving the recorded file an appropriate format wav format
  with speech_recog.AudioFile(wavfile) as source:
        audio_listened = rec.listen(source) ##Launching the API
  try:
    print("I think you said: \n" + rec.recognize_google(audio_listened))
    transcript = rec.recognize_google(audio_listened)
  except Exception as e:
    print(e)
  
  return verifyVoice(wavfile), transcript

In [ ]:
launchClientTranscript()

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
I think you said: 
hello
['wav']


('wav', 'hello')

# **TO DO (1)**

1. Test this google api transcription for voices in French if everything goes oni goes on proper.

2. Implement the google translate to convert French records into English transcripted from voice

In [ ]:
##Implement code